<a href="https://colab.research.google.com/github/GioLomia/Robot_Surface_Classifier/blob/master/Carear_Con_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from google.colab import files
import pandas as pd

import keras

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!ls
!kaggle competitions list

adc.json  sample_data
ref                                                deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                   2030-01-01 00:00:00  Getting Started  Knowledge       2635            True  
titanic                                            2030-01-01 00:00:00  Getting Started  Knowledge      10865           False  
house-prices-advanced-regression-techniques        2030-01-01 00:00:00  Getting Started  Knowledge       4391           False  
imagenet-object-localization-challenge             2029-12-31 07:00:00  Research         Knowledge         34           False  
competitive-data-science-predict-future-sales      2019-12-31 23:59:00  Playground           Kudos       2660           False  
two-sigma-financial-news                           2019-07-15 23:59:00  Featured  

In [0]:
!kaggle competitions files -c career-con-2019
!ls
!kaggle competitions download  -c career-con-2019 -p /content/kaggle
!mkdir root
%cd kaggle
!ls
!unzip /content/kaggle/X_train.csv.zip
!unzip /content/kaggle/X_test.csv.zip
!ls

name                   size  creationDate         
---------------------  ----  -------------------  
X_train.csv            47MB  2019-03-14 02:38:48  
y_train.csv            64KB  2019-03-14 02:38:22  
sample_submission.csv  51KB  2019-03-14 02:38:22  
X_test.csv             47MB  2019-03-14 02:38:48  
adc.json  sample_data
 64% 11.0M/17.1M [00:00<00:00, 54.5MB/s]
100% 17.1M/17.1M [00:00<00:00, 66.2MB/s]
  0% 0.00/64.1k [00:00<?, ?B/s]
100% 64.1k/64.1k [00:00<00:00, 49.8MB/s]
  0% 0.00/51.1k [00:00<?, ?B/s]
100% 51.1k/51.1k [00:00<00:00, 30.1MB/s]
 29% 5.00M/17.2M [00:00<00:00, 28.7MB/s]
100% 17.2M/17.2M [00:00<00:00, 66.3MB/s]
/content/kaggle
sample_submission.csv  X_test.csv.zip  X_train.csv.zip	y_train.csv
Archive:  /content/kaggle/X_train.csv.zip
  inflating: X_train.csv             
Archive:  /content/kaggle/X_test.csv.zip
  inflating: X_test.csv              
sample_submission.csv  X_test.csv.zip  X_train.csv.zip
X_test.csv	       X_train.csv     y_train.csv


In [0]:
df=pd.read_csv("/content/kaggle/X_train.csv")
y_train=pd.read_csv("/content/kaggle/y_train.csv")
df_test=pd.read_csv("/content/kaggle/X_test.csv")

In [0]:
df.iloc[:,4]+df.iloc[:,2:]

In [0]:
y_train

In [0]:
#####################FEATURE CREATION#####################
#Coordinate Features
df["X_Y_disp"]=df.orientation_X - df.orientation_Y
df["X_Z_disp"]=df.orientation_X - df.orientation_Z
df["X_W_disp"]=df.orientation_X - df.orientation_W
df["Y_Z_disp"]=df.orientation_Y - df.orientation_Z
df["Y_W_disp"]=df.orientation_Y - df.orientation_W
df["Z_W_disp"]=df.orientation_Z - df.orientation_W

df["X_Y_dist"]=((df.orientation_X - df.orientation_Y)**2)**0.5
df["X_Z_dist"]=((df.orientation_X - df.orientation_Z)**2)**0.5
df["X_W_dist"]=((df.orientation_X - df.orientation_W)**2)**0.5
df["Y_Z_dist"]=((df.orientation_Y - df.orientation_Z)**2)**0.5
df["Y_W_dist"]=((df.orientation_Y - df.orientation_W)**2)**0.5
df["Z_W_dist"]=((df.orientation_Z - df.orientation_W)**2)**0.5

#Velocity Features
df["abs_ang_vel"]=((df.angular_velocity_X**2)**0.5 + (df.angular_velocity_Y**2)**0.5 + (df.angular_velocity_Z**2)**0.5)
df["abs_ang_vel_dif"]=((df.angular_velocity_X - df.angular_velocity_Y - df.angular_velocity_Z)**2)**0.5
df["total_ang_vel_sum"]=df.angular_velocity_X + df.angular_velocity_Y + df.angular_velocity_Z
df["total_ang_vel_dif"]=df.angular_velocity_X - df.angular_velocity_Y - df.angular_velocity_Z

df["X_Y_ang_vel"]=df.angular_velocity_X - df.angular_velocity_Y
df["X_Z_ang_vel"]=df.angular_velocity_X - df.angular_velocity_Z
df["Y_Z_ang_vel"]=df.angular_velocity_Y - df.angular_velocity_Z

df["X_Y_ang_vel_abs"]=((df.angular_velocity_X - df.angular_velocity_Y)**2)**0.5
df["X_Z_ang_vel_abs"]=((df.angular_velocity_X - df.angular_velocity_Z)**2)**0.5
df["Y_Z_ang_vel_abs"]=((df.angular_velocity_Y - df.angular_velocity_Z)**2)**0.5

#Linear Acceleration Features
df["abs_lin_acc"]=((df.linear_acceleration_X**2)**0.5 + (df.linear_acceleration_Y**2)**0.5 + (df.linear_acceleration_Z**2)**0.5)
df["abs_lin_acc_dif"]=((df.linear_acceleration_X - df.linear_acceleration_Y - df.linear_acceleration_Z)**2)**0.5
df["total_lin_acc_sum"]=df.linear_acceleration_X + df.linear_acceleration_Y + df.linear_acceleration_Z
df["total_lin_acc_dif"]=df.linear_acceleration_X - df.linear_acceleration_Y - df.linear_acceleration_Z

df["X_Y_lin_acc"]=df.linear_acceleration_X - df.linear_acceleration_Y
df["X_Z_lin_acc"]=df.linear_acceleration_X - df.linear_acceleration_Z
df["Y_Z_lin_acc"]=df.linear_acceleration_Y - df.linear_acceleration_Z

df["X_Y_lin_acc_abs"]=((df.linear_acceleration_X - df.linear_acceleration_Y)**2)**0.5
df["X_Z_lin_acc_abs"]=((df.linear_acceleration_X - df.linear_acceleration_Z)**2)**0.5
df["Y_Z_lin_acc_abs"]=((df.linear_acceleration_Y - df.linear_acceleration_Z)**2)**0.5


In [0]:

    
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

def fe(actual):
    new = pd.DataFrame()
    actual['total_angular_velocity'] = (actual['angular_velocity_X'] ** 2 + actual['angular_velocity_Y'] ** 2 + actual['angular_velocity_Z'] ** 2) ** 0.5
    actual['total_linear_acceleration'] = (actual['linear_acceleration_X'] ** 2 + actual['linear_acceleration_Y'] ** 2 + actual['linear_acceleration_Z'] ** 2) ** 0.5
    
    actual['acc_vs_vel'] = actual['total_linear_acceleration'] / actual['total_angular_velocity']
    
    x, y, z, w = actual['orientation_X'].tolist(), actual['orientation_Y'].tolist(), actual['orientation_Z'].tolist(), actual['orientation_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    
    actual['total_angle'] = (actual['euler_x'] ** 2 + actual['euler_y'] ** 2 + actual['euler_z'] ** 2) ** 5
    actual['angle_vs_acc'] = actual['total_angle'] / actual['total_linear_acceleration']
    actual['angle_vs_vel'] = actual['total_angle'] / actual['total_angular_velocity']

    return actual

In [0]:
df=fe(df)

In [0]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

In [0]:
scaled_df=pd.DataFrame(scaled)
scaled_df.columns=df.columns

In [0]:
new_df=scaled_df

In [0]:
dist_feature=[]

for i in range(487679):
    dist_feature.append((((df.orientation_X[i]-df.orientation_X[i+1])**2)+((df.orientation_Y[i]-df.orientation_Y[i+1]))**2+
                        ((df.orientation_Z[i]-df.orientation_Z[i+1])**2)+((df.orientation_W[i]-df.orientation_W[i+1])**2)))



In [0]:
new_df

In [0]:
dist_feature.append(0)
import math
for i in dist_feature:
    i=math.sqrt(i)

In [0]:
df["abs_dist_traveled"]=dist_feature

In [0]:
compressed=[]
i=0
while(i<len(new_df)):
    appender=[]
    for j in range(128):
        appender.append(np.array(new_df.iloc[i+j]))
    compressed.append(np.array(appender))
    i=i+128

In [0]:
len(compressed[0][0])

55

In [0]:
experiment

In [0]:
resp_vec=list(y_train.surface)
print(resp_vec)

['fine_concrete', 'concrete', 'concrete', 'concrete', 'soft_tiles', 'tiled', 'soft_pvc', 'concrete', 'hard_tiles_large_space', 'tiled', 'soft_pvc', 'tiled', 'carpet', 'carpet', 'concrete', 'carpet', 'carpet', 'soft_tiles', 'tiled', 'soft_tiles', 'tiled', 'hard_tiles_large_space', 'soft_tiles', 'tiled', 'soft_tiles', 'soft_tiles', 'fine_concrete', 'hard_tiles', 'tiled', 'hard_tiles_large_space', 'soft_tiles', 'soft_tiles', 'fine_concrete', 'concrete', 'concrete', 'concrete', 'tiled', 'carpet', 'soft_pvc', 'wood', 'fine_concrete', 'wood', 'fine_concrete', 'wood', 'soft_tiles', 'hard_tiles', 'soft_pvc', 'fine_concrete', 'concrete', 'soft_pvc', 'concrete', 'soft_tiles', 'soft_tiles', 'concrete', 'carpet', 'soft_tiles', 'fine_concrete', 'fine_concrete', 'carpet', 'soft_tiles', 'concrete', 'tiled', 'soft_pvc', 'hard_tiles_large_space', 'soft_pvc', 'concrete', 'tiled', 'tiled', 'concrete', 'soft_pvc', 'concrete', 'concrete', 'soft_pvc', 'soft_pvc', 'wood', 'fine_concrete', 'soft_tiles', 'fine

In [0]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
enc.fit(resp_vec)
encoded_y=enc.transform(resp_vec)

In [0]:
from keras.utils import np_utils
encoded_y
dummy_y = np_utils.to_categorical(encoded_y)

In [0]:
len(x_train[0])

128

In [0]:
target=dummy_y

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(units=32,input_shape=(128,55),activation="relu",recurrent_dropout=0.3,dropout=0.2))
model.add(keras.layers.LSTM(64,input_shape=(3810,128,55),activation="relu"))
model.add(keras.layers.Dense(units=390,activation="relu"))

model.add(keras.layers.Dense(units=240,activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(units=9,activation="softmax"))

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model

In [0]:
model.fit(x_train,dummy_y,epochs=50,validation_split=0.2)